# Notebook 6 : MongoDB

In [ ]:
# Décommenter la ligne suivante pour installer pymongo
# %pip install pymongo

In [ ]:
import json

import pandas as pd
import pymongo

client = pymongo.MongoClient(
    # Coller ici la configuration donnée dans Onyxia
)

db = client.defaultdb

## Planètes de Star Wars

Nous considérons ici les données des planètes de *Star Wars* exportées à la fin du *Notebook 4*. Le fichier `planets.json` est également disponible dans le dossier des jeux de données.

1. Accéder à une collection `planets` et s'assurer qu'elle est vide grâce à la méthode `count_documents`.

In [ ]:
planets = db["planets"]
if planets.count_documents({}) > 0:
    # La collection n'est pas vide, drop supprime tous les documents
    planets.drop()

2. Importer les données des planètes dans la collection `planets`.

In [ ]:
with open("data/planets.json") as f:
    for planet_document in f.readlines():
        planets.insert_one(json.loads(planet_document))

print(f"{planets.count_documents({})} planètes")

3. Exporter l'ensemble des planètes sans l'identifiant `_id` dans un dataframe à l'aide du résultat de la méthode `find`.

In [ ]:
(
    pd.DataFrame(
        planets.find(projection={"_id": False})
    )
    .head()
)

4. Rechercher les planètes dont la période de rotation est égale à 25. Quel est le problème ? Combien y en a-t-il ?

In [ ]:
# Il ne semble y avoir aucune planète avec une période de rotation égale à 25
print(f"'rotation_period': 25 ----> {planets.count_documents({'rotation_period': 25})}")

# Il faut remarquer que les données sont des chaînes de caractères (ce sera corrigé dans la suite)
print(f"'rotation_period': '25' --> {planets.count_documents({'rotation_period': '25'})}")

(
    pd.DataFrame(
        planets.find(
            filter={"rotation_period": "25"},
            projection={"_id": False},
        )
    )
)

5. Même question mais en limitant la réponse aux clés `name`, `rotation_period`, `orbital_period` et `diameter`.

In [ ]:
(
    pd.DataFrame(
        planets.find(
            filter={"rotation_period": "25"},
            projection={
                "_id": False,
                "name": True,
                "rotation_period": True,
                "orbital_period": True,
                "diameter": True,
            },
        )
    )
)

6. Trier les planètes du résultat précédent par diamètre décroissant. Quel est le problème ?

In [ ]:
resultat = (
    pd.DataFrame(
        planets.find(
            filter={"rotation_period": "25"},
            projection={
                "_id": False,
                "name": True,
                "rotation_period": True,
                "orbital_period": True,
                "diameter": True,
            },
            sort=[("diameter", pymongo.DESCENDING)]
        )
    )
)

# Le tri n'est pas correct car la variable diameter est une chaîne de caractères
print(resultat.convert_dtypes().dtypes)

# Le tri est donc alphabétique, ce qui n'est pas ce que nous voulons
resultat

7. Vider la collection et importer à nouveau les données mais en faisant les corrections suivantes au préalable (un dataframe intermédiaire pourra être utilisé pour manipuler les données avant leur insertion) :
- convertir les valeurs numériques (gérer les cas `unknown`),
- supprimer les variables `created`, `edited`, `films`, `gravity`, `residents` et `url`.
- transformer les variables `climate` et `terrain` en listes de chaînes de caractères plutôt qu'une longue chaîne séparée par des virgules.

In [ ]:
# Chargement des données dans un dataframe intermédiaire
df_planets = pd.read_json("data/planets.json", lines=True)

# Conversion des valeurs numériques
numeric_columns = ["diameter", "orbital_period", "population", "rotation_period", "surface_water"]
df_planets[numeric_columns] = (
    df_planets[numeric_columns]
    .replace("unknown", pd.NA) # Replace les "unknown" par des valeurs manquantes
    .apply(pd.to_numeric) # Une façon de convertir en numériques
)

# Suppression des colonnes
df_planets.drop(
    columns=["created", "edited", "films", "gravity", "residents", "url"],
    inplace=True # Modifie le dataframe
)

# Transformation en listes de chaînes de caractères
columns = ["climate", "terrain"]
df_planets[columns] = (
    df_planets[columns]
    # Séparateur ", " pour diviser les longues chaînes de caractères
    .apply(lambda serie: serie.str.split(", "))
)

# Les données sont prêtes, nous pouvons vider la collection pour insérer les nouvelles versions

planet_documents = (
    df_planets
    .to_json(orient="records", lines=True) # Documents séparés par des retours à la ligne "\n"
    .split("\n") # Séparateur "\n" pour diviser la liste des documents
)

planets.drop() # Suppression des documents de la collection
planets.insert_many( # Insertion des nouvelles versions
    [
        json.loads(planet_document)
        for planet_document in planet_documents
        if planet_document != "" # Évite le dernier retour à la ligne
    ]
)

# Affichage du résultat
(
    pd.DataFrame(
        planets.find(projection={"_id": False})
    )
    .head()
)

8. Reprendre la question 6 et vérifier que le résultat est maintenant correct.

In [ ]:
# Le tri est maintenant dans l'ordre numérique
(
    pd.DataFrame(
        planets.find(
            filter={"rotation_period": 25}, # La valeur n'est plus une chaîne de caractères grâce à la question précédente
            projection={
                "_id": False,
                "name": True,
                "rotation_period": True,
                "orbital_period": True,
                "diameter": True,
            },
            sort=[("diameter", pymongo.DESCENDING)]
        )
    )
)

9. Extraire les planètes dont le nom commence par `T`.

In [ ]:
pd.DataFrame(
    planets.find(
        filter={"name": {"$regex": "^T"}},
        projection={"_id": False}
    )
)

10. Extraire les planètes dont le diamètre est strictement supérieur à 10000 et où se trouvent des montagnes.

In [ ]:
pd.DataFrame(
    planets.find(
        filter={
            "$and": [
                # Opérateur $gt pour 'strictement supérieur'
                {"diameter": {"$gt": 10000}},
                # Opérateur $in pour tester l'inclusion (grâce à notre nouvelle version)
                {"terrain": {"$in": ["mountains"]}},
            ]
        },
        projection={"_id": False}
    )
)

11. Rechercher puis supprimer la planète dont le nom est `unknown`.

In [ ]:
# Recherche de la planète unknown
pd.DataFrame(
    planets.find(
        filter={"name": "unknown"},
        projection={"_id": False},
    )
)

In [ ]:
# Suppression de la planète unknown
print(f"Avant : {planets.count_documents({})} planètes")
planets.delete_one({"name": "unknown"})
print(f"Après : {planets.count_documents({})} planètes")

12. Mettre en œuvre un pipeline d'agrégation qui calcule le nombre de planètes dans la collection. Verifier le résultat avec la méthode `count_documents`.

In [ ]:
print(f"Avec count_documents : {planets.count_documents ({})}")

pd.DataFrame(
    planets.aggregate(
        [
            {"$group": {"_id": None, "count": {"$sum": 1}}},
        ]
    )
)

13. Mettre en œuvre un pipeline d'agrégation pour calculer le diamètre moyen et la somme des populations des planètes contenant des glaciers.

In [ ]:
pd.DataFrame(
    planets.aggregate(
        [
            {
                "$match": {
                    "terrain": {"$in": ["glaciers"]}
                }
            },
            {
                "$group": {
                    "_id": None,
                    "diameter": {"$avg": "$diameter"},
                    "population": {"$sum": "$population"},
                }
            },
        ]
    )
)